In [ ]:
import os
os.chdir('..')

import pandas as pd
import numpy as np
import torch

from prettytable import PrettyTable

from recpack.preprocessing.preprocessors import DataFramePreprocessor
from recpack.preprocessing.filters import Deduplicate, MinRating, MinItemsPerUser
from recpack.scenarios import WeakGeneralization

from hyperopt import fmin, tpe, hp

# helpers & metrics
from src.helper_functions.data_formatting import *
from src.helper_functions.metrics_accuracy import *
from src.helper_functions.metrics_coverage import *
from src.helper_functions.metrics_exposure import *

# models
from src.recommenders.ease import myEASE
from src.recommenders.slim_bn import BNSLIM
from src.recommenders.fslr import FSLR
from src.recommenders.slim_bn_admm import BNSLIM_ADMM
from src.recommenders.mf_fair import FairMF
from src.recommenders.fda import FDA_bpr

import json
import itertools
import time
import pickle

import matplotlib.pyplot as plt

In [ ]:
plays = (
    pd.read_csv(
        "lastfm-1k/userid-timestamp-artid-artname-traid-traname.tsv",
        sep="\t",
        header=None,
        usecols=[0, 2],
        names=["User_id", "Artist_id"], # unique pairs
        on_bad_lines="skip",
    )
    .groupby(["User_id", "Artist_id"])
    .size() # number of times a user has played songs by a particular artist
    .reset_index(name="Plays")
    .merge(
        pd.read_csv(
            "lastfm-1k/userid-profile.tsv",
            sep="\t",
            usecols=[0, 2], # targeting User_id and Age columns
            names=["User_id", "Age"],
            skiprows=1,
        ),
        on="User_id",
        how="left",
    )
    .dropna(subset=["Age"])
)

# filter plays dataframe based on age constraint
plays = plays[(plays["Age"] >= 10) & (plays["Age"] <= 90)]

plays = plays[plays["Plays"] > plays.groupby("User_id")["Plays"].transform("mean")]
plays["Plays"] = 1

# set threshold for age groups
age_threshold = 30
plays["Age_Group"] = (plays["Age"] > age_threshold).astype(int)

with open("lastfm-1k/lastfm1k_artists_mbid.pkl", 'rb') as f: 
    artists = pickle.load(f)
with open("lastfm-1k/lastfm1k_genres_filtered_final.pkl", 'rb') as f: 
    genres = pickle.load(f)

# process artists data
artists = (
    pd.DataFrame({"Artist_id": artists, "Genres": list(genres.values())})
    .assign(Genres=lambda df: df["Genres"].apply(lambda x: ", ".join(x)).str.split(", "))
    .explode("Genres")
    .replace("NA", np.nan)
    .dropna(subset=["Genres"])
)

# filter the dataframe to only include rows with these genres
artists = artists[artists["Genres"].isin(["country", "folk", "metal", "indie", "jazz", "hip hop"])]
# merge together
plays = pd.merge(plays, artists, on="Artist_id", how="inner")

In [ ]:
plays.head()

In [ ]:
plays_pp = DataFramePreprocessor("Artist_id", "User_id")

# define filters
deduplicate = Deduplicate("Artist_id", "User_id")
min_items_per_user_filter = MinItemsPerUser(10, "Artist_id", "User_id")

# add filters to pre-processor
plays_pp.add_filter(deduplicate)
plays_pp.add_filter(min_items_per_user_filter)

# create interaction matrix object
im = plays_pp.process(plays)

# create genre to artist id dictionary (before applying deduplication filtering)
raw_genre_dict = plays.groupby("Genres")["Artist_id"].apply(lambda x: list(set(x))).to_dict()

# apply filters to plays frame directly
plays = min_items_per_user_filter.apply(deduplicate.apply(plays))

# genre - inner iids dictionary
inner_genre_dict = {
    genre: get_inner_item_ids(plays_pp, raw_iids)
    for genre, raw_iids in raw_genre_dict.items()
    }

In [ ]:
# compute sparsity after filtering
sparsity = 1 - im.density

# calculate user interaction and item popularity ranges
user_interactions = im.binary_values.sum(axis=1)
item_popularities = im.binary_values.sum(axis=0)
print(f"User interaction ranges from {user_interactions.min()} to {user_interactions.max()}. Item popularity ranges from {item_popularities.min()} to {item_popularities.max()}.")

# get the raw ids of all users involved
raw_uids = get_raw_user_ids(plays_pp, im.active_users)

# create uid - age group mapping df
age_mapping_df = plays[plays["User_id"].isin(raw_uids)][["User_id", "Age_Group"]].drop_duplicates()

# get the raw/inner ids of all users involved that are above 30 (a30)
raw_uids_a30 = age_mapping_df.loc[age_mapping_df["Age_Group"] == 1, "User_id"].to_numpy()
inner_uids_a30 = get_inner_user_ids(plays_pp, raw_uids_a30)

# get the raw/inner ids of all users involved that are below 30 (b30)
raw_uids_b30 = age_mapping_df.loc[age_mapping_df["Age_Group"] == 0, "User_id"].to_numpy()
inner_uids_b30 = get_inner_user_ids(plays_pp, raw_uids_b30)

num_interactions_a30, num_interactions_b30 = im.binary_values[inner_uids_a30].sum(), im.binary_values[inner_uids_b30].sum()

# table stats
statTable1 = PrettyTable(["data set","|U|","|I|","int(I)","sparsity"])
statTable1.add_row(["LastFM", str(im.num_active_users), str(im.num_active_items), str(im.num_interactions), str(round(sparsity*100,2))])
print(statTable1)

statTable2 = PrettyTable(["data set","attribute","|Age > 30|","int(Age > 30)","|Age ≤ 30|","int(Age ≤ 30)"])
statTable2.add_row(["LastFM", "age", str(len(raw_uids_a30)), str(num_interactions_a30), str(len(raw_uids_b30)), str(num_interactions_b30)])
print(statTable2)

In [ ]:
# Sum the number of plays for each user within each genre and age group
user_genre_age_sum = plays.groupby(["User_id", "Genres", "Age_Group"])["Plays"].sum().reset_index()

# Calculate the average number of plays per user for each genre and age group
genre_age_avg_plays = user_genre_age_sum.groupby(["Genres", "Age_Group"])["Plays"].mean().reset_index()

# Pivot the data for plotting
genre_age_avg_plays_pivot = genre_age_avg_plays.pivot(index="Genres", columns="Age_Group", values="Plays").fillna(0)

plt.figure(figsize=(15, 10))
genre_age_avg_plays_pivot.plot(kind="bar", figsize=(15,10))

plt.title("Average Number of Plays per User by Genre and Age Group")
plt.xlabel("Genre")
plt.ylabel("Average Number of Plays per User")
plt.legend([f"Age ≤ {age_threshold}", f"Age > {age_threshold}"], title="Age Group")
plt.tight_layout()
plt.show()

## Experiments

In [ ]:
# define K for Top-K
K = 10

# Define alpha, the parameter that balances the importance of NDCG and Equity in the objective function.
# Setting alpha = 0.5 gives equal weight to both metrics, aiming to balance relevance (NDCG) and fairness (Equity).
# Adjusting alpha allows for prioritizing one metric over the other.
# For instance, setting alpha closer to 1.0 would prioritize NDCG (accuracy), while setting it closer to 0.0 would prioritize Equity (fairness).
alpha = 0.2

# define seed; seeds tested (1452, 1994, 42, 7, 13800)
SEED = 7

# define scenario
# Note: Due to the nature of the utilized algorithms (User-User neighborhood methods), 
# only scenarios that include the 'validation in' set in the 'validation training' set, 
# and the 'test in' set in the 'full training' set, are applicable.
scenario = WeakGeneralization(validation=True, seed=SEED)
scenario.split(im)

# define time threshold
SECONDS = 24*3600

# define number of evaluations
EVALUATIONS = 50

In [ ]:
def accuracy_objective(model, fit_args={}):
    model.fit(scenario.validation_training_data.binary_values, **fit_args)

    # generate predictions and mask training interactions
    predictions = model.predict(scenario.validation_training_data.binary_values).toarray()
    predictions[scenario.validation_training_data.binary_values.nonzero()] = -np.inf

    ndcg, _ = tndcg_at_n(predictions, scenario.validation_data_out.binary_values, K)

    return 1-ndcg

def combined_objective(model, fit_args={}):
    model.fit(scenario.validation_training_data.binary_values, **fit_args)

    if "users_features" in fit_args: #fda
        predictions = model.model_.predict().toarray()
    else:
        predictions = model.predict(scenario.validation_training_data.binary_values).toarray()
    predictions[scenario.validation_training_data.binary_values.nonzero()] = -np.inf

    ndcg, _ = tndcg_at_n(predictions, scenario.validation_data_out.binary_values, K)
    equity, _, _ = c_equity_at_n(predictions[inner_uids_a30, :], predictions[inner_uids_b30, :], inner_genre_dict, K)

    return alpha * (1-ndcg) + (1 - alpha) * equity

In [ ]:
# for fairmf
sst_field = torch.zeros((im.num_active_users, im.num_active_items), dtype=torch.bool)
sst_field[inner_uids_a30, :] = True

# for fda
users_features = np.zeros(im.num_active_users); users_features[inner_uids_b30] = 1

In [ ]:
# optimize ease
optimisation_results_ease = fmin(
    fn=lambda param: accuracy_objective(myEASE(l2=param["l2"], method="user")),
    space={"l2": hp.loguniform("l2", np.log(1e0), np.log(1e4))},
    algo=tpe.suggest,
    timeout = SECONDS,
    max_evals = EVALUATIONS,
)

# optimize bnslim
optimisation_results_bnslim = fmin(
    fn=lambda param: combined_objective(BNSLIM(knn=100, l1=param["l1"], l2=param["l2"], l3=param["l3"], method="user", seed=SEED), {"inner_ids_npr": inner_uids_b30}),
    space={"l1": hp.loguniform("l1", np.log(1e-3), np.log(7)),
           "l2": hp.loguniform("l2", np.log(1e-3), np.log(7)),
           "l3": hp.loguniform("l3", np.log(1e1), np.log(1e4))
           }, 
    algo=tpe.suggest,
    timeout=SECONDS,
    max_evals=EVALUATIONS
)

# optimize fslr
optimisation_results_fslr = fmin(
    fn=lambda param: combined_objective(FSLR(l1=param["l1"], l2=param["l2"], method="user"), {"inner_ids_pr": inner_uids_a30, "inner_ids_npr": inner_uids_b30}),
    space={"l1": hp.loguniform("l1", np.log(1e-3), np.log(1e1)),
           "l2": hp.loguniform("l2", np.log(1e0), np.log(1e4))},
    algo=tpe.suggest,
    timeout=SECONDS,
    max_evals=EVALUATIONS
)

# optimize bnslim admm
optimisation_results_bnslim_admm = fmin(
    fn=lambda param: combined_objective(BNSLIM_ADMM(l1=param["l1"], l2=param["l2"], l3=param["l3"], method="user"), {"inner_ids_npr": inner_uids_b30}),
    space={"l1": hp.loguniform("l1", np.log(1e-3), np.log(50)),
           "l2": hp.loguniform("l2", np.log(1e0), np.log(1e4)),
           "l3": hp.loguniform("l3", np.log(1e-3), np.log(1e3))},
    algo=tpe.suggest,
    timeout = SECONDS,
    max_evals = EVALUATIONS,
)

# optimize FairMF
factor_choices = [32, 64, 128]
optimisation_results_fairmf = fmin(
    fn=lambda param: combined_objective(FairMF(batch_size=im.num_active_users, learning_rate=param["learning_rate"], l2=param["l2"], num_factors=param["num_factors"], seed=SEED), {"sst_field": sst_field}),
    space={"learning_rate": hp.loguniform("learning_rate", np.log(1e-6), np.log(1e0)),
           "l2": hp.loguniform("l2", np.log(1e-6), np.log(1e-1)),
           "num_factors": hp.choice("num_factors", factor_choices)
           },
    algo=tpe.suggest,
    timeout=SECONDS,
    max_evals=EVALUATIONS
)

optimisation_results_fairmf["num_factors"] = factor_choices[optimisation_results_fairmf["num_factors"]]

# optimize FDA
# define the parameter choices
num_ng_choices = [5, 7, 9, 10]
ratio_choices = [0.1, 0.3, 0.5, 0.7]
all_combinations = itertools.product(num_ng_choices, ratio_choices)

best_params = None
best_score = float("inf")

for num_ng, noise_ratio in all_combinations:
    score = combined_objective(
        FDA_bpr(num_ng=num_ng, noise_ratio=noise_ratio, seed=SEED), 
        {"users_features": users_features}
    )

    if score < best_score:
        best_score = score
        best_params = {"num_ng": num_ng, "noise_ratio": noise_ratio}

opt_params = {}
opt_params.update({
    "ease": optimisation_results_ease,
    "bnslim": optimisation_results_bnslim,
    "fslr": optimisation_results_fslr,
    "bnslim_admm": optimisation_results_bnslim_admm,
    "fairmf": optimisation_results_fairmf,
    "fda": best_params
})

folder = f"lastfm-1k/{SEED}"; os.makedirs(folder, exist_ok=True)
with open(f"{folder}/opt_params.json", "w") as f: json.dump(opt_params, f, indent=4)

In [ ]:
with open(f"lastfm-1k/{SEED}/opt_params.json", "r") as f: opt_params = json.load(f)

def initialize_models(opt_params):
    return {
        "ease": myEASE(l2=opt_params["ease"]["l2"], method="user"),
        "bnslim": BNSLIM(knn=100, l1=opt_params["bnslim"]["l1"], l2=opt_params["bnslim"]["l2"], l3=opt_params["bnslim"]["l3"], maxIter=50, method="user", seed=SEED),
        "fslr": FSLR(l1=opt_params["fslr"]["l1"], l2=opt_params["fslr"]["l2"], method="user"),
        "bnslim_admm": BNSLIM_ADMM(l1=opt_params["bnslim_admm"]["l1"], l2=opt_params["bnslim_admm"]["l2"], l3=opt_params["bnslim_admm"]["l3"], method="user"),
        "fairmf": FairMF(batch_size=im.num_active_users, l2=opt_params["fairmf"]["l2"], learning_rate=opt_params["fairmf"]["learning_rate"], num_factors=opt_params["fairmf"]["num_factors"], seed=SEED),
        "fda": FDA_bpr(
            noise_ratio=opt_params["fda"]["noise_ratio"], 
            num_ng=opt_params["fda"]["num_ng"],
            seed=SEED
        )
    }

# initialize models
models = initialize_models(opt_params)

# define the models, list sizes, and metrics
list_sizes = [10, 20, 50, 100]
metrics = ["ndcg", "recall", "c-equity", "u-parity"]

# initialize a dictionary to store results with mean and standard deviation
results = {
    "iters_num": {model: 0 for model in ["bnslim", "fslr", "bnslim_admm", "fairmf"]},
    "fit_time": {model: 0 for model in models.keys()},
    **{metric: {model: {size: {"mean": 0, "std": 0} for size in list_sizes} for model in models.keys()} for metric in metrics}
}

In [ ]:
for model_name, model in models.items():
    
    print(f"Training model {model_name}...")

    params = {}
    if model_name == "fslr":
        params = {"inner_ids_pr": inner_uids_a30, "inner_ids_npr": inner_uids_b30}
    elif model_name in ["bnslim", "bnslim_admm"]:
        params = {"inner_ids_npr": inner_uids_b30}
    elif model_name == "fairmf":
        params = {"sst_field": sst_field}
    elif model_name == "fda":
        params = {"users_features": users_features}
    
    start_time = time.time()
    model.fit(scenario.full_training_data.binary_values, **params)
    results["fit_time"][model_name] = time.time() - start_time

    if model_name in results["iters_num"]:
        if model_name == "fairmf":
            results["iters_num"][model_name] = model.epochs
        else:
            results["iters_num"][model_name] = model.iters

    # generate predictions and mask training interactions
    if model_name == "fda":
        y_pred = model.model_.predict()
    else:
        y_pred = model.predict(scenario.full_training_data.binary_values)
    predictions = y_pred.toarray()
    predictions[scenario.full_training_data.binary_values.nonzero()] = -np.inf

    # compute evaluation metrics for different values of K
    for K in list_sizes:
        # accuracy metrics
        results["ndcg"][model_name][K]["mean"], results["ndcg"][model_name][K]["std"] = tndcg_at_n(predictions, scenario.test_data_out.binary_values, K)
        results["recall"][model_name][K]["mean"], results["recall"][model_name][K]["std"] = recall_at_n(predictions, scenario.test_data_out.binary_values, K)

        # fairness metrics
        results["c-equity"][model_name][K]["mean"], results["c-equity"][model_name][K]["std"], _ = c_equity_at_n(predictions[inner_uids_a30, :], predictions[inner_uids_b30, :], inner_genre_dict, K)

        protected = np.ones(im.num_active_users); protected[inner_uids_b30] = 0
        results["u-parity"][model_name][K]["mean"], results["u-parity"][model_name][K]["std"] = u_parity_at_n(predictions, protected, inner_genre_dict, K)

    # # save model
    # pickle.dump(model, open(f"lastfm-1k/{SEED}/{model_name}.pkl", "wb"))

# save results
with open(f"lastfm-1k/{SEED}/results.json", "w") as f: json.dump(results, f, indent=4)